In [132]:
import pandas as pd
import tweepy
import json
import os
from database import *
from songkick import *
import math
import re
import requests

In [57]:
# # to run on heroku
# # Consumer keys and access tokens, used for OAuth
# songkick_api_key = os.environ['songkick_api_key']
# tw_consumer_key = os.environ["tw_consumer_key"]
# tw_consumer_secret = os.environ["tw_consumer_secret"]
# tw_access_token = os.environ["tw_access_token"]
# tw_access_token_secret = os.environ["tw_access_token_secret"]
# database_url = os.environ['HEROKU_POSTGRESQL_IVORY_URL']
# heroku_app_name =  os.environ['heroku_app_name']
# table_name =  os.environ['table_name']

# to run locally
keys={}
with open(os.path.abspath("keys.json"),"r") as f:
    keys = json.loads(f.read())
songkick_api_key = keys['songkick_api_key']
tw_consumer_key = keys["tw_consumer_key"]
tw_consumer_secret = keys["tw_consumer_secret"]
tw_access_token = keys["tw_access_token"]
tw_access_token_secret = keys["tw_access_token_secret"]
database_url = keys['HEROKU_POSTGRESQL_IVORY_URL']
heroku_app_name =  keys['heroku_app_name']
table_name =  keys['table_name']

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(tw_consumer_key, tw_consumer_secret)
auth.set_access_token(tw_access_token, tw_access_token_secret)
 
# Creation of the actual interface, using authentication
api = tweepy.API(auth)

In [58]:
table = readTable()

In [59]:
table.head(20)

,artistId,artistName,concertTime,content,eventDate,billingIndex,tweeted
0,10036759,Haley Fohr,2019-12-22 04:31:59-05:00,Haley Fohr is playing their first show in DC! ...,2019-12-05,2,1
1,9505769,Clarice Jensen,2019-12-22 04:32:00-05:00,Clarice Jensen is playing their first show in ...,2019-12-05,1,1
2,9380054,Another Sky,2019-12-22 04:32:00-05:00,Another Sky is playing their first show in DC!...,2019-12-05,1,1
3,10099589,New Order of Existence,2019-12-22 11:31:58-05:00,New Order of Existence is playing their first ...,2019-12-05,3,1
4,2068564,Nate Brown,2019-12-22 12:01:59-05:00,Nate Brown is playing their first show in DC! ...,2019-12-05,2,1
5,7469949,Kicking Sunrise,2019-12-22 12:31:59-05:00,Kicking Sunrise is playing their first show in...,2019-12-05,2,1
6,9238404,So Sensitive,2019-12-22 12:31:59-05:00,So Sensitive is playing their first show in DC...,2019-12-05,2,1
7,8871489,Chris Renzema,2019-12-22 12:32:00-05:00,Chris Renzema is playing their first show in D...,2019-12-05,1,1
8,74265,Nathan Brown,2019-12-22 14:01:58-05:00,Nathan Brown is playing their first show in DC...,2019-12-05,3,1
9,315359,Eric Schenkman,2019-12-23 04:32:00-05:00,Eric Schenkman is playing their first show in ...,2019-12-06,1,1


In [163]:
print(findHandle('Some Kind of Nightmare'))

somekindofnightmare
#####
screen name: prezbharrison
name: benjaminharrison
followers: +18
total score: 18
----
None


In [161]:
def formatName(name):
    name = name.lower()
    name = name.replace(" ", "")
    name = name.replace("_", "")
    name = name.replace("'", "")
    name = name.replace("\"", "")
    name = name.replace("(US)", "")
    name = name.replace(",", "")
    name = name.replace("(", "")
    name = name.replace(")", "")
    name = re.sub('\W', '', name)
    return name

def findHandle(artistName):

    searchAccounts = api.search_users(artistName)

    artistName = formatName(artistName)
    print(artistName)
    print("#####")

    maxScore = 0
    maxHandle = "None"

    for acj in searchAccounts:
        thisScore = 0
        account = acj._json
#         print(account.keys())
        thisScreenName = formatName(account['screen_name'])
        thisName = formatName(account['name'])

        print("screen name: " + thisScreenName)
        print("name: " + thisName)

        # check name
        if (artistName == thisName):
            print("name perfect match +50")
            thisScore = thisScore + 50
        elif (artistName in thisName):
            print("name sub match +25")
            thisScore = thisScore + 25

        if ("band" in thisName):
            print("name band match +30")
            thisScore = thisScore + 30

        if ("music" in thisName):
            print("name music match +30")
            thisScore = thisScore + 30

        # check screen name
        if (artistName == thisScreenName):
            print("screen name perfect match +25")
            thisScore = thisScore + 25
        elif (artistName in thisScreenName):
            print("screen name sub match +13")
            thisScore = thisScore + 13

        if ("band" in thisScreenName):
            print("screen name band match +15")
            thisScore = thisScore + 15

        if ("music" in thisScreenName):
            print("screen name music match +15")
            thisScore = thisScore + 15

        # check verified
        if (account['verified'] == True):
            print("verified +20")
            thisScore = thisScore + 20

        # check followers
        followersPoints = math.ceil(account['followers_count']/100)
        if (followersPoints > 20):
            followersPoints = 20
        thisScore = thisScore + followersPoints
        print("followers: +" + str(followersPoints))

        # if they have an image
        if (account['default_profile_image']):
            print("no background image: -50")
            thisScore = thisScore - 50
            
        # see if they have a url
        if (account['url'] != None):
            r = requests.head(account['url'], allow_redirects=True)
            accountLink = r.url.lower()
            musicSites = ['soundcloud''bandcamp','spotify','itunes']
            for musicSite in musicSites:
                if musicSite in accountLink:
                    print("music site link " + musicSite + ": +50")
                    thisScore = thisScore + 50
                    
            if (artistName in accountLink):
                print("site link " + artistName + ": +30")
                thisScore = thisScore + 30
            print(r.url)

        # private/public
        if (account['protected']):
            print("protected: -50")
            thisScore = thisScore - 50

        # check description

        description = account['description'].lower()
        descriptionWordList = ['mgmt', 'managment','band','inquiries','music','tour','album','stream','song','single']
        for word in descriptionWordList:
            if word in description:
                print("description match " + word + ": +20")
                thisScore = thisScore + 20

        # look for fan accounts
        if ("fan account" in description):
            print("fan account: -100")
            thisScore = thisScore - 100

  

        print("total score: " + str(thisScore))
        print("----")


  
        if (thisScore > maxScore):
            maxScore = thisScore
            maxIndex = thisIndex
            maxHandle = account['screen_name']
    
    if (maxScore > 99):
        return maxHandle
    else:
        return "None"                        